In [1]:
import pandas as pd
import sqlite3

pd.options.mode.chained_assignment = None  # default='warn'


df = pd.read_csv("rowData.csv")
# main manipulations
df['id'] = df.index
#df.drop("name",axis=1, inplace=True)
df[['english_market','arabic_market']] = df['marketName'].str.split(' - ', 1, expand=True)
df = df[['id' ,"name","arabic_market","english_market","price","scientificName","companies","pic" , "details"]]
df['companies'] = df['companies'].str.replace('\n','')
df.rename(columns={'scientificName': 'scientific_name' , "pic" : "picture"}, inplace=True)
arabicEnd = df.id[df.arabic_market.isnull()].iloc[0] 
value = df.loc[0:arabicEnd,:]['english_market'] ;
df.loc[0:arabicEnd,:]['name'] = value
df.drop(['arabic_market' , "english_market"],axis=1, inplace=True)
df.head(1)

,id,name,price,scientific_name,companies,picture,details
0,0,Human albumin 20 % behring (low salt),545.0,Human albumin,"Csl behering-gmbh,2s pharma group,",https://dalilaldwaa.com/data/medicine/15183.png,حول الألبومين البشري\r\nمادة نقل جزء بروتين ال...
1,1,Hexgabalin 75 mg 30 caps,111.0,Pregabalin,"Global napi pharmaceuticals,",https://dalilaldwaa.com/data/medicine/703.png,Hexgabalinis أشار إلى إدارة آلام الأعصاب. الأل...
2,2,Hexgabalin 150mg 30 caps,120.0,Pregabalin,"Global napi pharmaceuticals,",https://dalilaldwaa.com/data/medicine/783.png,Hexgabalinis أشار إلى إدارة آلام الأعصاب. الأل...
3,3,Histotec 8 mg 30 f.c. tab,15.0,Betahistine,"Eipico,",https://dalilaldwaa.com/data/medicine/11569.png,تكوين\r\nكل قرص مغلفة يحتوي على:\r\nBetahisti...
4,4,Histotec 16mg 20 f.c. tab,19.0,Betahistine,"Eipico,",https://dalilaldwaa.com/data/medicine/3338.png,المكونات\r\nكل قرص مغلفة الفيلم يحتوي على:\r\n...


In [ ]:
df2 = pd.read_excel("IOUO_OO.mdb.xls")
df2.rename(columns={'اسم الدواء': 'name' , "السعر" : "price"}, inplace=True)
df2.drop(df2[df2.name.str.replace(" ","").duplicated()].index, axis=0, inplace=True)

dropIndex = []
found = 0

for drug in df2.iterrows() :
    keyWords = drug[1][0].split(" ")
    new = False 
    for word in keyWords :
        try :
            if (df['name'].str.upper().str.contains(" {} ".format(word.upper())).sum()) == 0 : # not found
                new = True
                break
        except :
            pass
        
    if new :
        found +=1 
    else :
        dropIndex.append(drug[0])
       
print("end with {}".format(found))
df2.drop(dropIndex , axis=0, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2['id'] = df2.index + df.shape[0]
df2.tail(10) 

In [4]:
#save

merged = pd.concat([df, df2])
merged['favorite'] = False


conn = sqlite3.connect('testSql.db')
merged.to_sql('data', conn, if_exists='replace', index=False)

conn.execute('''CREATE TABLE IF NOT EXISTS "orders" (
	"id"	TEXT,
	"price"	TEXT,
	"itemsCount"	INTEGER,
	"imageCount"	INTEGER,
    "done" INTEGER
    "time"	TEXT,
);''')

conn.execute('''CREATE TABLE IF NOT EXISTS "messages" (
	"id"	INTEGER,
	"text"	TEXT,
	"time"	TEXT,
    "sender" INTEGER
);''')

conn.commit()

conn.close()

merged.to_csv("data.csv", encoding='utf-8' ,index=False)